<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p data
!find /content/drive/My\ Drive/tcc/resultados/ -type f ! -name '*.gsheet' -exec cp {} data/ \;

In [ ]:
import os
import glob

def get_latest_file(directory, pattern):
    file_list = glob.glob(os.path.join(directory, pattern))

    if not file_list:
        raise FileNotFoundError("No files found matching the pattern.")

    latest_file = max(file_list, key=os.path.getmtime)

    return latest_file

    # return open(latest_file, 'r')



In [ ]:
directory = 'data/'
pattern = 'QA_ppc2024*.csv'


latest_file = get_latest_file(directory, pattern)
print(latest_file)

data/QA_ppc2024_01_09_2024_21-41-48.csv


In [ ]:
import pandas as pd
df = pd.read_csv(latest_file)

In [ ]:
df

,Pergunta,RespostaIdeal,RespostaGerada,ConsultaExpandida,ConfiguracaoDeInferencia
0,Qual é a ementa de Introdução à Computação?,Introdução ao Curso de Ciência da Computação e...,"A ementa do componente curricular ""Introdução ...",INTRODUÇÃO À COMPUTAÇÃO | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...
1,Qual é a ementa de Introdução ao Desenvolvimen...,"Introdução à Web: histórico, estrutura e funci...","A ementa de Introdução ao Desenvolvimento Web,...",INTRODUÇÃO AO DESENVOLVIMENTO WEB | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...
2,Qual é a ementa de Matemática C?,Grandezas proporcionais. Noções de geometria. ...,"A ementa de Matemática C, conforme o Plano Ped...",MATEMÁTICA C | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...
3,Qual é a ementa de Introdução à Filosofia?,A natureza e especificidade do discurso filosó...,"A ementa de Introdução à Filosofia é: ""A natur...",INTRODUÇÃO À FILOSOFIA | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...
4,Qual é a ementa de Algoritmos e Programação?,Conceito e construção de algoritmos. Tipos bás...,A ementa da disciplina de Algoritmos e Program...,ALGORITMOS E PROGRAMAÇÃO | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...
...,...,...,...,...,...
56,Quais laboratórios estão disponíveis para os a...,5 (cinco) laboratórios de Programação;1 (um) l...,Existem vários laboratórios disponíveis para o...,Quais laboratórios estão disponíveis para os a...,Mixtral e Gemma 9b(Transformação). Contexto: 5...
57,Quais são as atividades de extensão relacionad...,As atividades de extensão relacionadas ao curs...,As atividades de extensão relacionadas ao curs...,Atividades de extensão relacionadas ao curso \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...
58,Por que mudou o PPC?,O PPC (Plano Pedagógico do Curso) de Ciência d...,"A resposta para a pergunta ""Por que mudou o PP...",Por que mudou o Plano Pedagógico do Curso? \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...
59,Quais são as formas de ingresso disponíveis pa...,"O acesso aos cursos de graduação da UFFS, tant...",De acordo com o Plano Pedagógico do Curso (PPC...,Quais são as formas de ingresso disponíveis pa...,Mixtral e Gemma 9b(Transformação). Contexto: 5...


In [ ]:
from google.colab import userdata


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from enum import Enum



class Correcao(str, Enum):
    TOTALMENTE_CORRETO = "TOTALMENTE_CORRETO"
    PARCIALMENTE_CORRETO = "PARCIALMENTE_CORRETO"
    TOTALMENTE_INCORRETO = "TOTALMENTE_INCORRETO"


class Avaliacao(BaseModel):
    nota: int = Field(description="A nota da avaliação (de 0 a 10)", ge=0, le=10)
    justificativa: str = Field(description="Uma breve justificativa para a nota")

class AvaliacaoGeral(BaseModel):
    avaliacao_geral: str = Field(description="A avaliação/comentário geral")
    exatidao_factual: Avaliacao = Field(description=" Verifique se a resposta gerada está correta em termos de fatos (fatos = RESPOSTA FATO).Esse quesito é crucial. Não exite em dar notas realmente baixas caso a resposta esteja incorreta. Atribua uma nota de 0 a 10.")
    completude: Avaliacao = Field(description="Verifique se a resposta cobre todos os pontos importantes presentes na ground truth. Esse quesito é crucial. Não exite em dar notas realmente baixas caso a resposta esteja incorreta. Atribua uma nota de 0 a 10.")
    relevancia: Avaliacao = Field(description="Determine se a resposta gerada é relevante e se mantém fiel ao contexto e à ground truth. Esse quesito é crucial. Não exite em dar notas realmente baixas caso a resposta esteja incorreta. Atribua uma nota de 0 a 10.")
    correcao: Correcao = Field(description="Indica o quão correta foi a resposta gerada em termos gerais, em relação a resposta fato.")


In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(api_key = userdata.get('GROQ_API_KEY'))
structured_llm = model.with_structured_output(AvaliacaoGeral)

In [ ]:
structured_llm.invoke(query)

AvaliacaoGeral(avaliacao_geral="A ementa do componente curricular 'Introdução à Computação' não está explicitamente fornecida no trecho do Plano Pedagógico do Curso (PPC) que me foi fornecido. No entanto, podemos inferir o conteúdo dessa disciplina a partir do seu nome e da análise dos créditos e horas associadas a ela.", exatidao_factual=Avaliacao(nota=7, justificativa='A resposta é uma inferência baseada nas informações disponíveis, mas não é uma informação exata ou fática sobre o conteúdo da disciplina.'), completude=Avaliacao(nota=8, justificativa="A resposta fornece uma análise detalhada da informação disponível no PPC sobre a disciplina 'Introdução à Computação', incluindo a duração, créditos e distribuição de aulas."), relevancia=Avaliacao(nota=9, justificativa="A resposta é relevante para a pergunta, proporcionando informações úteis sobre o possível conteúdo da disciplina 'Introdução à Computação'."))

In [ ]:
from groq import Groq

SYSTEM_PROMPT = """
Dado o seguinte contexto e a resposta gerada, compare-a com a ground truth fornecida. Sua tarefa é avaliar, de maneira rígida,  se a resposta gerada reflete com precisão as informações da ground truth. Use uma abordagem passo a passo, analisando os seguintes aspectos:

1. **Exatidão factual**: Verifique se a resposta gerada está correta em termos de fatos (RESPOSTA FATO) . Esse quesito é crucial. Não exite em dar notas realmente baixas caso a resposta esteja incorreta. Atribua uma nota de 0 a 10.
2. **Completude**: Verifique se a resposta cobre todos os pontos importantes presentes na ground truth. Atribua uma nota de 0 a 10.
3. **Relevância**: Determine se a resposta gerada é relevante e se mantém fiel ao contexto e à ground truth. Atribua uma nota de 0 a 10.
4. **Correção**: Indique o quão correta foi a resposta gerada em termos gerais, em relação a resposta fato.

OBS:
Avalie todos os campos com rigor. Verifique se a resposta aborda todos os pontos da ground truth e se mantém fiel ao contexto.

Por fim, retorne um JSON contendo as notas e uma breve justificativa para cada aspecto avaliado.



Com base no contexto fornecido, na resposta gerada e na ground truth, aqui está a avaliação:

```json
{

  "avaliacao_geral": [Insira a avaliação/comentário aqui],
  "correcao": [Insira a correção aqui],
  "exatidao_factual": {
    "nota": [Insira a nota aqui],
    "justificativa": "[Insira uma breve justificativa aqui]"
  },
  "completude": {
    "nota": [Insira a nota aqui],
    "justificativa": "[Insira uma breve justificativa aqui]"
  },
  "relevancia": {
    "nota": [Insira a nota aqui],
    "justificativa": "[Insira uma breve justificativa aqui]"
  }
}

"""


def build_query(response, goal_response):
  return f"""
  [RESPOSTA GERADA]
  {response}
  [FIM RESPOSTA GERADA]
  [RESPOSTA FATO]
  {goal_response}
  [FIM RESPOSTA FATO]
  """

def get_model_name(model_id):
    return model_id.split('-')[0].capitalize()



In [ ]:
import json

client = Groq(api_key = userdata.get('GROQ_API_KEY'))
def get_completion(query, model_id = 'llama3-groq-70b-8192-tool-use-preview'):

  completion = client.chat.completions.create(
      model=model_id,
      messages=[
          {
              "role": "system",
              "content": SYSTEM_PROMPT
          },
          {
              "role": "user",
              "content": query
          }
      ],
      temperature=0.5,
      max_tokens=1024,
      top_p=1,
      stream=False,
      response_format={"type": "json_object"},
      stop=None,
  )

  return json.loads(completion.choices[0].message.content)





In [ ]:
from langchain_groq import ChatGroq

def get_evaluation(query, model_id='llama3-groq-70b-8192-tool-use-preview', count=0):
    try:
        model = ChatGroq(model=model_id, api_key=userdata.get('GROQ_API_KEY'))
        structured_llm = model.with_structured_output(AvaliacaoGeral)
        return structured_llm.invoke(query)
    except Exception as e:
        if count < 3:
            print(f"Erro ao tentar invocar o modelo: {e}. Tentativa {count + 1} de 3.")
            return get_evaluation(query, model_id, count + 1)

        print("Falha em todas as tentativas. Retornando valor padrão.")
        return AvaliacaoGeral(
            avaliacao_geral="Avaliação não pôde ser realizada.",
            exatidao_factual=Avaliacao(nota=0, justificativa="Não foi possível avaliar."),
            completude=Avaliacao(nota=0, justificativa="Não foi possível avaliar."),
            relevancia=Avaliacao(nota=0, justificativa="Não foi possível avaliar."),
            correcao=Correcao.TOTALMENTE_INCORRETO
        )


In [ ]:
df.head(1)

,Pergunta,RespostaIdeal,RespostaGerada,ConsultaExpandida,ConfiguracaoDeInferencia
0,Qual é a ementa de Introdução à Computação?,Introdução ao Curso de Ciência da Computação e...,"A ementa do componente curricular ""Introdução ...",INTRODUÇÃO À COMPUTAÇÃO | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...


In [ ]:
response, goal_response  = df.iloc[59]['RespostaGerada'], df.iloc[59]['RespostaIdeal']


query = build_query(response, goal_response)

response = get_evaluation(query, 'llama-3.1-70b-versatile')
response

AvaliacaoGeral(avaliacao_geral='A resposta gerada apresenta algumas informações relevantes sobre os turnos de ingresso para o curso de Ciência da Computação da UFFS, mas não aborda as formas de ingresso disponíveis de forma precisa e completa em relação à resposta fato. A resposta gerada apenas menciona o turno vespertino e noturno, mas não detalha as outras formas de ingresso disponíveis.', exatidao_factual=Avaliacao(nota=6, justificativa='A resposta gerada apresenta algumas informações corretas sobre os turnos de ingresso, mas não é precisa em relação às formas de ingresso disponíveis.'), completude=Avaliacao(nota=4, justificativa='A resposta gerada não cobre todos os pontos importantes presentes na resposta fato, como as diferentes formas de ingresso disponíveis.'), relevancia=Avaliacao(nota=7, justificativa='A resposta gerada é relevante para o contexto da pergunta, mas não é completa em relação à resposta fato.'), correcao=<Correcao.PARCIALMENTE_CORRETO: 'PARCIALMENTE_CORRETO'>)

In [ ]:
response.correcao.value

'PARCIALMENTE_CORRETO'

In [ ]:
response.exatidao_factual.nota

7

In [ ]:
def apply_evaluation(row, model_id):
    query = build_query(row['RespostaGerada'], row['RespostaIdeal'])
    eval = get_evaluation(query, model_id)
    model_name = get_model_name(model_id)

    evaluation_results = {
        f'{model_name}_Nota_Exatidao_Factual': eval.exatidao_factual.nota,
        f'{model_name}_Justificativa_Exatidao_Factual': eval.exatidao_factual.justificativa,
        f'{model_name}_Nota_Completude': eval.completude.nota,
        f'{model_name}_Justificativa_Completude': eval.completude.justificativa,
        f'{model_name}_Nota_Relevancia': eval.relevancia.nota,
        f'{model_name}_Justificativa_Relevancia': eval.relevancia.justificativa,
        f'{model_name}_Justificativa_Geral': eval.avaliacao_geral,
        f'{model_name}_Correcao': eval.correcao.value
    }

    mean_eval = (
        evaluation_results[f'{model_name}_Nota_Exatidao_Factual'] +
        evaluation_results[f'{model_name}_Nota_Completude'] +
        evaluation_results[f'{model_name}_Nota_Relevancia']
    ) / 3


    evaluation_results[f'{model_name}_Media'] = mean_eval

    print(evaluation_results)

    return pd.Series(evaluation_results)

In [ ]:
from time import sleep

model_id = 'llama-3.1-70b-versatile'
subset_df = df.sample(n=3)

results = []
for index, row in subset_df.iterrows():
    evaluation_results = apply_evaluation(row, model_id)
    for key, value in evaluation_results.items():
        subset_df.at[index, key] = value
    sleep(2)


subset_df.to_csv('avaliacoes.csv', index=False)

{'Llama_Nota_Exatidao_Factual': 7, 'Llama_Justificativa_Exatidao_Factual': 'A resposta apresenta informações factuais corretas, mas não está completa.', 'Llama_Nota_Completude': 6, 'Llama_Justificativa_Completude': 'A resposta apresenta uma grade curricular detalhada, mas não está completa devido a limitações de caracteres.', 'Llama_Nota_Relevancia': 8, 'Llama_Justificativa_Relevancia': 'A resposta é relevante e mantém fiel ao contexto e à ground truth.', 'Llama_Justificativa_Geral': 'A resposta gerada apresenta uma grade curricular detalhada, mas não está completa devido a limitações de caracteres. A referência ao PPC é útil, mas a resposta poderia ser mais concisa e objetiva.', 'Llama_Correcao': 'PARCIALMENTE_CORRETO', 'Llama_Media': 7.0}
{'Llama_Nota_Exatidao_Factual': 10, 'Llama_Justificativa_Exatidao_Factual': 'A resposta está correta em termos de fatos.', 'Llama_Nota_Completude': 10, 'Llama_Justificativa_Completude': 'A resposta cobre todos os pontos importantes presentes na grou

In [ ]:
from time import sleep

model_id = 'llama-3.1-70b-versatile'

results = []
for index, row in df.iterrows():
    evaluation_results = apply_evaluation(row, model_id)
    for key, value in evaluation_results.items():
        df.at[index, key] = value
    sleep(2)


df.to_csv('avaliacoes.csv', index=False)

{'Llama_Nota_Exatidao_Factual': 7, 'Llama_Justificativa_Exatidao_Factual': 'A resposta gerada apresenta alguns fatos corretos sobre a disciplina, mas também contém informações não precisas e suposições.', 'Llama_Nota_Completude': 6, 'Llama_Justificativa_Completude': 'A resposta gerada não aborda todos os tópicos mencionados na resposta fato, como a introdução ao curso e à universidade, e os comandos básicos em sistemas operacionais abertos.', 'Llama_Nota_Relevancia': 8, 'Llama_Justificativa_Relevancia': 'A resposta gerada está relacionada ao tópico da disciplina e fornece uma visão geral sobre os conceitos básicos da computação.', 'Llama_Justificativa_Geral': 'A resposta gerada fornece uma visão geral sobre os conceitos básicos da disciplina, mas não está completamente de acordo com a resposta fato fornecida.', 'Llama_Correcao': 'PARCIALMENTE_CORRETO', 'Llama_Media': 7.0}
{'Llama_Nota_Exatidao_Factual': 10, 'Llama_Justificativa_Exatidao_Factual': 'A resposta gerada coincide com a respo

In [ ]:
from time import sleep

model_id = 'mixtral-8x7b-32768'

results = []
for index, row in df.iterrows():
    evaluation_results = apply_evaluation(row, model_id)
    for key, value in evaluation_results.items():
        df.at[index, key] = value
    sleep(8

          )


df.to_csv('avaliacoes.csv', index=False)

{'Mixtral_Nota_Exatidao_Factual': 6, 'Mixtral_Justificativa_Exatidao_Factual': 'A resposta gerada é parcialmente correta, pois fornece uma visão geral do possível conteúdo da disciplina, mas não fornece informações específicas sobre a ementa.', 'Mixtral_Nota_Completude': 7, 'Mixtral_Justificativa_Completude': 'A resposta gerada aborda a maioria dos aspectos importantes, mas não fornece informações específicas sobre a ementa da disciplina.', 'Mixtral_Nota_Relevancia': 8, 'Mixtral_Justificativa_Relevancia': 'A resposta gerada é relevante, pois fornece informações que podem ajudar o usuário a entender o possível conteúdo da disciplina.', 'Mixtral_Justificativa_Geral': "A resposta gerada fornece uma boa visão geral do possível conteúdo da disciplina 'Introdução à Computação', mas não fornece informações específicas sobre a ementa.", 'Mixtral_Correcao': 'PARCIALMENTE_CORRETO', 'Mixtral_Media': 7.0}
{'Mixtral_Nota_Exatidao_Factual': 10, 'Mixtral_Justificativa_Exatidao_Factual': 'A resposta e

In [ ]:
df.head()

,Pergunta,RespostaIdeal,RespostaGerada,ConsultaExpandida,ConfiguracaoDeInferencia,Llama_Nota_Exatidao_Factual,Llama_Justificativa_Exatidao_Factual,Llama_Nota_Completude,Llama_Justificativa_Completude,Llama_Nota_Relevancia,...,Llama_Media,Mixtral_Nota_Exatidao_Factual,Mixtral_Justificativa_Exatidao_Factual,Mixtral_Nota_Completude,Mixtral_Justificativa_Completude,Mixtral_Nota_Relevancia,Mixtral_Justificativa_Relevancia,Mixtral_Justificativa_Geral,Mixtral_Correcao,Mixtral_Media
0,Qual é a ementa de Introdução à Computação?,Introdução ao Curso de Ciência da Computação e...,"A ementa do componente curricular ""Introdução ...",INTRODUÇÃO À COMPUTAÇÃO | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...,7.0,A resposta gerada apresenta alguns fatos corre...,6.0,A resposta gerada não aborda todos os tópicos ...,8.0,...,7.000000,6.0,"A resposta gerada é parcialmente correta, pois...",7.0,A resposta gerada aborda a maioria dos aspecto...,8.0,"A resposta gerada é relevante, pois fornece in...",A resposta gerada fornece uma boa visão geral ...,PARCIALMENTE_CORRETO,7.000000
1,Qual é a ementa de Introdução ao Desenvolvimen...,"Introdução à Web: histórico, estrutura e funci...","A ementa de Introdução ao Desenvolvimento Web,...",INTRODUÇÃO AO DESENVOLVIMENTO WEB | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...,10.0,A resposta gerada coincide com a resposta fato.,9.0,"A resposta apresenta a ementa completa, mas nã...",10.0,...,9.666667,10.0,A resposta está totalmente de acordo com a gro...,9.0,A resposta abrange todos os aspectos relevante...,10.0,A resposta é altamente relevante e mantém-se f...,"A resposta está bem estruturada, clara e infor...",TOTALMENTE_CORRETO,9.666667
2,Qual é a ementa de Matemática C?,Grandezas proporcionais. Noções de geometria. ...,"A ementa de Matemática C, conforme o Plano Ped...",MATEMÁTICA C | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...,10.0,A resposta gerada contém os mesmos tópicos de ...,8.0,A resposta gerada menciona todos os pontos imp...,9.0,...,9.000000,10.0,A resposta fato é exata e corresponde à ementa...,10.0,A resposta abrange todos os tópicos da ementa ...,10.0,A resposta é altamente relevante e mantém-se f...,A resposta estava correta e bem detalhada.,TOTALMENTE_CORRETO,10.000000
3,Qual é a ementa de Introdução à Filosofia?,A natureza e especificidade do discurso filosó...,"A ementa de Introdução à Filosofia é: ""A natur...",INTRODUÇÃO À FILOSOFIA | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...,10.0,A resposta gerada está correta em termos de fa...,10.0,A resposta gerada cobre todos os pontos import...,10.0,...,10.000000,10.0,A resposta apresenta a ementa do curso exatame...,7.0,"A resposta fornece a ementa do curso, mas omit...",8.0,"A resposta fornece uma informação relevante, m...",A resposta está parcialmente correta. A ementa...,PARCIALMENTE_CORRETO,8.333333
4,Qual é a ementa de Algoritmos e Programação?,Conceito e construção de algoritmos. Tipos bás...,A ementa da disciplina de Algoritmos e Program...,ALGORITMOS E PROGRAMAÇÃO | EMENTA \n,Mixtral e Gemma 9b(Transformação). Contexto: 5...,10.0,A resposta gerada está correta em termos de fa...,10.0,A resposta gerada apresenta todos os pontos im...,10.0,...,10.000000,10.0,A resposta gerada é exatamente correspondente ...,10.0,A resposta gerada é completa e coberto todos o...,10.0,A resposta gerada é altamente relevante e mant...,"A resposta gerada é clara e informativa, cobri...",TOTALMENTE_CORRETO,10.000000


In [ ]:
latest_file.split('/')[1]

'QA_ppc2024_01_09_2024_21-41-48.csv'

In [ ]:
import shutil
import os

def save_csv_to_drive(csv_file_path, drive_folder_path, drive_file_name):
    # Create the directory if it doesn't exist
    os.makedirs(drive_folder_path, exist_ok=True)

    # Construct the full path for the source file
    source_file_path = f'/content/{csv_file_path}'  # Assuming the file is in Colab's root directory

    drive_path = f'{drive_folder_path}/{os.path.basename(drive_file_name)}'

    # Copy the file
    shutil.copy(source_file_path, drive_path)
    print(f'File saved to: {drive_path}')

In [ ]:
csv_file_path = 'avaliacoes.csv'
drive_folder_path = '/content/drive/MyDrive/tcc/resultados/validados'
save_csv_to_drive(csv_file_path, drive_folder_path,latest_file.split('/')[1])

File saved to: /content/drive/MyDrive/tcc/resultados/validados/QA_ppc2024_01_09_2024_21-41-48.csv
